In [1]:
import pandas as pd
import numpy as np
import nltk
from numpy.linalg import norm

In [2]:
CORPUS = pd.read_csv("data/CORPUS.csv")

In [3]:
CORPUS

,book_num,chap_num,para_num,sent_num,token_num,token_str,term_str,pos,pos_group
0,0,1,0,0,0,When,when,WRB,WR
1,0,1,0,0,1,I,i,PRP,PR
2,0,1,0,0,2,wake,wake,VBP,VB
3,0,1,0,0,3,up,up,RB,RB
4,0,1,0,0,5,the,the,DT,DT
...,...,...,...,...,...,...,...,...,...
643772,5,57,122,1,3,mended,mended,VBN,VB
643773,5,57,122,2,0,We,we,PRP,PR
643774,5,57,122,2,1,mend,mend,VBP,VB
643775,5,57,122,2,2,each,each,DT,DT


In [4]:
VOCAB = CORPUS.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['max_pos'] = CORPUS[['term_str','pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['max_pos_group'] = CORPUS[['term_str','pos_group']].value_counts().unstack(fill_value=0).idxmax(1)
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1
VOCAB['stop'] = VOCAB.index.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['porter_stem'] = VOCAB.apply(lambda x: stemmer1.stem(x.name), 1)

In [5]:
BOW = CORPUS.groupby(['book_num', 'chap_num']+['term_str']).term_str.count().to_frame('n')
DTCM = BOW.n.unstack(fill_value=0)
DF = DTCM.astype('bool').sum()
N = DTCM.shape[0]
IDF = np.log2(N / DF)

In [6]:
dfidf = pd.DataFrame(DF*IDF)

In [7]:
VOCAB['dfidf'] = dfidf

In [8]:
VOCAB

,n,n_chars,p,i,max_pos,max_pos_group,stop,porter_stem,dfidf
term_str,,,,,,,,,
00,10,2,0.000016,15.974273,CD,CD,0,00,31.295624
05,1,2,0.000002,19.296202,CD,CD,0,05,7.800900
080712,1,6,0.000002,19.296202,CD,CD,0,080712,7.800900
1,25,1,0.000039,14.652345,CD,CD,0,1,50.591249
100,1,3,0.000002,19.296202,CD,CD,0,100,7.800900
...,...,...,...,...,...,...,...,...,...
zoe,56,3,0.000087,13.488847,NNP,NN,0,zoe,41.678774
zone,7,4,0.000011,16.488847,NN,NN,0,zone,34.954815
zones,3,5,0.000005,17.711239,NNS,NN,0,zone,18.647812


In [9]:
VOCAB.columns

Index(['n', 'n_chars', 'p', 'i', 'max_pos', 'max_pos_group', 'stop',
       'porter_stem', 'dfidf'],
      dtype='object')

In [10]:
VOCAB.sort_values(by='dfidf', ascending=False).head(20)

,n,n_chars,p,i,max_pos,max_pos_group,stop,porter_stem,dfidf
term_str,,,,,,,,,
catch,126,5,0.000196,12.318922,VB,VB,0,catch,118.354527
having,121,6,0.000188,12.377338,VBG,VB,1,have,118.354527
somewhere,117,9,0.000182,12.425837,RB,RB,0,somewher,118.354527
ones,121,4,0.000188,12.377338,NNS,NN,0,one,118.354527
lift,112,4,0.000174,12.488847,VBP,VB,0,lift,118.354527
true,138,4,0.000214,12.187677,JJ,JJ,0,true,118.354527
went,116,4,0.000180,12.438221,VBD,VB,0,went,118.354527
moving,122,6,0.000190,12.365464,VBG,VB,0,move,118.354527
deep,117,4,0.000182,12.425837,JJ,JJ,0,deep,118.354527


In [11]:
BOW

n
book_num chap_num term_str      
0        1        a          126
                  abandoned    1
                  abernathy    1
                  abounds      1
                  about       13
...                          ...
5        57       young        3
                  younger      1
                  your         5
                  zeke        22
                  zip          3

[188846 rows x 1 columns]

In [12]:
DTCM

term_str           00  05  080712  1  100  15  150pound  17  18  2  ...  \
book_num chap_num                                                   ...   
0        1          0   0       0  0    0   0         0   0   0  0  ...   
         2          0   0       0  0    0   0         0   0   0  0  ...   
         3          0   0       0  0    0   0         0   0   0  0  ...   
         4          0   0       0  0    0   0         0   0   0  0  ...   
         5          0   0       0  0    0   0         0   0   0  0  ...   
...                ..  ..     ... ..  ...  ..       ...  ..  .. ..  ...   
5        53         0   0       0  0    0   0         0   0   0  0  ...   
         54         0   0       0  0    0   0         0   0   0  0  ...   
         55         0   0       0  0    0   0         0   0   0  0  ...   
         56         0   0       0  0    0   0         0   0   0  0  ...   
         57         0   0       0  0    0   0         0   0   0  0  ...   

term_str           zipper  zippers  zipping  zips  zling  zoe  zone  zones  \
book_num chap_num                                                            
0        1              0        0        0     0      0    0     0      0   
         2              0        0        0     1      0    0     0      0   
         3              0        0        0     0      0    0     0      0   
         4              0        0        0     0      0    0     0      0   
         5              0        0        0     0      1    0     0      0   
...                   ...      ...      ...   ...    ...  ...   ...    ...   
5        53             0        0        0     0      0    0     0      0   
         54             0        0        0     0      0    0     0      0   
         55             0        0        0     0      0    0     0      0   
         56             0        0        0     0      0    0     0      0   
         57             0        0        0     0      0    0     0      0   

term_str           zoom  zooms  
book_num chap_num               
0        1            0      0  
         2            0      0  
         3            0      0  
         4            0      0  
         5            0      0  
...                 ...    ...  
5        53           0      0  
         54           0      0  
         55           0      0  
         56           0      0  
         57           0      0  

[223 rows x 18121 columns]

In [13]:
TF = DTCM.T / DTCM.T.max()
TF = TF.T

In [14]:
TF

term_str            00   05  080712    1  100   15  150pound   17   18    2  \
book_num chap_num                                                             
0        1         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         2         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         3         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         4         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         5         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
...                ...  ...     ...  ...  ...  ...       ...  ...  ...  ...   
5        53        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         54        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         55        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         56        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         57        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   

term_str           ...  zipper  zippers  zipping      zips     zling  zoe  \
book_num chap_num  ...                                                      
0        1         ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         2         ...     0.0      0.0      0.0  0.004545  0.000000  0.0   
         3         ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         4         ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         5         ...     0.0      0.0      0.0  0.000000  0.006173  0.0   
...                ...     ...      ...      ...       ...       ...  ...   
5        53        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         54        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         55        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         56        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         57        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   

term_str           zone  zones  zoom  zooms  
book_num chap_num                            
0        1          0.0    0.0   0.0    0.0  
         2          0.0    0.0   0.0    0.0  
         3          0.0    0.0   0.0    0.0  
         4          0.0    0.0   0.0    0.0  
         5          0.0    0.0   0.0    0.0  
...                 ...    ...   ...    ...  
5        53         0.0    0.0   0.0    0.0  
         54         0.0    0.0   0.0    0.0  
         55         0.0    0.0   0.0    0.0  
         56         0.0    0.0   0.0    0.0  
         57         0.0    0.0   0.0    0.0  

[223 rows x 18121 columns]

In [15]:
IDF

term_str
00        5.215937
05        7.800900
080712    7.800900
1         4.215937
100       7.800900
            ...   
zoe       4.630975
zone      4.993545
zones     6.215937
zoom      6.800900
zooms     7.800900
Length: 18121, dtype: float64

In [16]:
TFIDF = TF*IDF

In [17]:
TFIDF

term_str            00   05  080712    1  100   15  150pound   17   18    2  \
book_num chap_num                                                             
0        1         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         2         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         3         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         4         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         5         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
...                ...  ...     ...  ...  ...  ...       ...  ...  ...  ...   
5        53        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         54        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         55        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         56        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         57        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   

term_str           ...  zipper  zippers  zipping      zips     zling  zoe  \
book_num chap_num  ...                                                      
0        1         ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         2         ...     0.0      0.0      0.0  0.030913  0.000000  0.0   
         3         ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         4         ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         5         ...     0.0      0.0      0.0  0.000000  0.048154  0.0   
...                ...     ...      ...      ...       ...       ...  ...   
5        53        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         54        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         55        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         56        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         57        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   

term_str           zone  zones  zoom  zooms  
book_num chap_num                            
0        1          0.0    0.0   0.0    0.0  
         2          0.0    0.0   0.0    0.0  
         3          0.0    0.0   0.0    0.0  
         4          0.0    0.0   0.0    0.0  
         5          0.0    0.0   0.0    0.0  
...                 ...    ...   ...    ...  
5        53         0.0    0.0   0.0    0.0  
         54         0.0    0.0   0.0    0.0  
         55         0.0    0.0   0.0    0.0  
         56         0.0    0.0   0.0    0.0  
         57         0.0    0.0   0.0    0.0  

[223 rows x 18121 columns]

In [18]:
TFIDF_L2 = TFIDF.apply(lambda x: x / norm(x), 1)

In [19]:
TFIDF_L2

term_str            00   05  080712    1  100   15  150pound   17   18    2  \
book_num chap_num                                                             
0        1         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         2         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         3         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         4         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         5         0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
...                ...  ...     ...  ...  ...  ...       ...  ...  ...  ...   
5        53        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         54        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         55        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         56        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   
         57        0.0  0.0     0.0  0.0  0.0  0.0       0.0  0.0  0.0  0.0   

term_str           ...  zipper  zippers  zipping      zips     zling  zoe  \
book_num chap_num  ...                                                      
0        1         ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         2         ...     0.0      0.0      0.0  0.038403  0.000000  0.0   
         3         ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         4         ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         5         ...     0.0      0.0      0.0  0.000000  0.041197  0.0   
...                ...     ...      ...      ...       ...       ...  ...   
5        53        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         54        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         55        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         56        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   
         57        ...     0.0      0.0      0.0  0.000000  0.000000  0.0   

term_str           zone  zones  zoom  zooms  
book_num chap_num                            
0        1          0.0    0.0   0.0    0.0  
         2          0.0    0.0   0.0    0.0  
         3          0.0    0.0   0.0    0.0  
         4          0.0    0.0   0.0    0.0  
         5          0.0    0.0   0.0    0.0  
...                 ...    ...   ...    ...  
5        53         0.0    0.0   0.0    0.0  
         54         0.0    0.0   0.0    0.0  
         55         0.0    0.0   0.0    0.0  
         56         0.0    0.0   0.0    0.0  
         57         0.0    0.0   0.0    0.0  

[223 rows x 18121 columns]

In [20]:
VOCAB.to_csv("data/VOCAB.csv")
BOW.to_csv("data/BOW.csv")
DTCM.to_csv("data/DTM.csv")
TFIDF.to_csv("data/TFIDF.csv")
TFIDF_L2.to_csv("data/TFIDF_L2.csv")